## Public Demo of IncQuery Server Jupyter Client Extensions

### Preliminaries

#### Setup IQS Connection

Execute the following piece of code to connect to the public IncQuery Server demo instance.

If you have non-guest privileges, uncomment the additional lines to specify your credentials.

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    #address='https://openmbee.incquery.io/api',
    #user='guest',
    #password='incqueryserverguest'
)

#### Select MMS commit to consider

Run the next code block to display the commit selector widget, and use it to browse around the MMS repository. When you've had your fun, make sure to leave it in a state where a commit is selected from the _IQS4MMS Demos_ org, as we have made sure to pre-index and load those commits in the IQS.

In [ ]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget()

The following piece of code assigns the Python name `model` to the MMS commit selected above, and checks whether the model is indeed indexed and loaded by IQS, which is required for the rest of the demo 

In [ ]:
model = commit_selector.value().to_model_compartment()
if model.is_loaded_by_server(iqs):
    print("We may proceed.")
else: 
    print("Model is not indexed&loaded by IQS, so the next demo steps will not work.")
    print(" (Unfortunately, guest users are not allowed to control model indexing.)")
    print("Please select another model from the 'IQS4MMS Demos' org.")

### Model validation

#### Perform validation checks with custom validation rules (progress with indexed models only)

Request and display a validation report for the selected model

In [ ]:
validation_report = iqs.validation.validate_model_compartment(model)
validation_report

The diagnostic results can also be simply processed in Python, as a list of diagnostic items:

In [ ]:
validation_report.to_list_of_diagnostic_items()

#### Visualize custom validation reports via Pandas dataframes and Plot.ly

The following block renders results in Pandas dataframe format, ready for complex client-side manipulation:

In [ ]:
validation_report_df = validation_report.to_data_frame()
validation_report_df

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [ ]:
import cufflinks as cf
cf.go_offline()

Preprocess results using Pandas and visualize the output using Plot.ly; this is handy for generating many kinds of custom validation reports.

First, here is a diagram showing the number of violations found for each validation rule:

In [ ]:
validation_report_df.constraint_element_name.value_counts().iplot( 
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_diagnostic_rules',
    yTitle='Number of Diagnostics per Diagnostic Rule', title='Rule Match Counts',
)

Next, we do some precomputation with Pandas and plot a chart of violating elements against their total number of violations; if there are multiple severities (e.g. warnings and errors), the bars will be subdivided (stacked bar chart).

In [ ]:
validation_report_grouped = validation_report_df.groupby(by=['matching_element_relative_id','severity'])
validation_report_unstacked = validation_report_grouped.size().unstack(level=-1, fill_value=0)
validation_report_unstacked['total'] = validation_report_unstacked.apply(sum, axis=1)
validation_report_unstacked = validation_report_unstacked.sort_values(by=['total'], ascending=False).drop('total', axis=1)

In [ ]:
validation_report_unstacked.iplot( 
    kind='bar', barmode='stack', filename='cufflinks/stacked-bar-chart/diagnostics_per_element',
    yTitle='Number of Diagnostics per Violating Element', title='Violations Per Element',
)

### Custom model queries

#### Execute pre-registered query and process results

See which queries are registered and ready for execution:

In [ ]:
iqs.queries.list_queries()

Execute a custom query registered on the server and display its results. 

Below we demonstrate a query that assigns a pair of in-house custom requirements coverage metrics to packages in the model. 

(Towards the end of the notebook we will see how such custom queries can be defined and registered on the server.)

In [ ]:
from iqs_jupyter import schema
qResults = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
      model_compartment = model,
      query_fqn = "iqs4mms.demo.coverage.packageCoverage"
    ))
qResults

The results can also be simply processed in Python:

In [ ]:
qResults.to_list_of_matches()

#### Extract individual model elements, execute queries with parameter bindings

Descriptors of individual model elements can be extracted into Python variables from query results...

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['pack']
first_result_element

...alternatively, model element descriptors can be directly constructed using element identifiers:

In [ ]:
some_element = model.get_element_in_compartment_by_id("_18_0_5_baa02e2_1454283816868_357127_158658") 
some_element

Model elements or simple values can be used as parameter bindings to restrict the requested results:

In [ ]:
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
        model_compartment = model,
        query_fqn = "iqs4mms.demo.coverage.packageCoverage",
        parameter_binding = binding(pack=some_element)
    ))
qResults_restricted

#### Visualize query results using Pandas dataframes and Plot.ly

The following block query renders results in Pandas dataframe format, ready for complex client-side manipulation:

In [ ]:
qResults_df = qResults.to_data_frame()
qResults_df

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [ ]:
import cufflinks as cf
cf.go_offline()

Preprocess results using Pandas and visualize the output using Plot.ly:

In [ ]:
qResults_df['pack'] = qResults_df['pack'].apply(lambda element: element.relative_element_id)
qResults_df.set_index('pack', inplace=True)
qResults_df

In [ ]:
qResults_df.iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/coverage',
    yTitle='Block in Package Traced to Requirements', title='Requirements Coverage by Package',
)

### Sandbox

In [ ]:
from iqs_jupyter import schema
iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
        model_compartment = model,
        query_fqn = "iqs4mms.demo.benchmark.requirements"
    ))

## Extra section for privileged users
Do not forget to specify your privileged credentials at the top of the notebook, in the first code cell

### Repository management, indexing commits

Force the server to refresh its knowledge of commits in the repository:

In [ ]:
iqs.mms_repository.update_mms_repository()

Index another model from the repository, and then load the index into server memory:

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

### Define and register custom ad-hoc queries

The custom model queries defined below will discover multiple forms of traceability to _Requirement_ elements, determine the coverage of _SysML_ _Blocks_, and aggregate coverage metrics for _Package_s containing these _Blocks_.

In [ ]:
coverage_query_package = "iqs4mms.demo.coverage"
coverage_query_main = "iqs4mms.demo.coverage.packageCoverage"
coverage_query_code = '''

// custom in-house definitions for strong and weak coverage

pattern stronglyCovered(element: NamedElement) {
    find sysml.businessRequirement_SatisfiedBy(_, element);
} or {
    find sysml.businessRequirement_TracedTo(_, element);
}
pattern weaklyCovered(element: NamedElement) {
    find stronglyCovered(otherElement);
    find coveragePropagates+(otherElement, element);
    neg find stronglyCovered(element);
}
pattern coveragePropagates(from: NamedElement, to: NamedElement) {
    // from container to part
    Class.ownedAttribute(from, part);
    Property.aggregation(part, ::composite);
    TypedElement.type(part, to);
} or { 
    // from general superclassifier / block to specific block
    Generalization.general(gen, from);
    Generalization.specific(gen, to);
}

// queries to assemble a report on (transitive) package contents

pattern blockInPackage(block: Class, pack: Package) {
    find sysml.Block(block, _);    
    Package.packagedElement+(pack, block);
}
pattern stronglyCoveredBlockInPackage(block: Class, pack: Package) {
    find stronglyCovered(block);    
    find blockInPackage(block, pack);
}
pattern weaklyCoveredBlockInPackage(block: Class, pack: Package) {
    find weaklyCovered(block);    
    find blockInPackage(block, pack);
}

pattern packageCoverage(
    pack: Package, 
    totalBlocks: java Integer, 
    stronglyCovered: java Integer, 
    weaklyCovered: java Integer
) {
    totalBlocks     == count find blockInPackage(_, pack);
    stronglyCovered == count find stronglyCoveredBlockInPackage(_, pack);
    weaklyCovered   == count find weaklyCoveredBlockInPackage(_, pack);
}

'''

Queries need to be registered on the server before they can be evaluated.

In [ ]:
if coverage_query_main not in iqs.queries.list_queries().query_fq_ns:
    # attempt to register query only if not already registered
    iqs.queries.register_queries_plain_text(coverage_query_code, query_package=coverage_query_package)
else:
    print("Query is already registered")    

### Sandbox